In [ ]:
import pandas as pd
import sqlite_utils
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import datetime

In [ ]:
def clean_fips(fips):
    if pd.isnull(fips):
        return "-1"
    else:
        return ("00000" + str(int(fips)))[-5:]

In [ ]:
# NYC FIPS CODES TO CONSOLIDATE INTO NYC GEOGRAPHY
NYC_FIPS = ["36061", "36005", "36081", "36047", "36085"]

In [ ]:
# 2018 POPULATION ESTIMATES
pop = pd.read_csv(
    "https://www.ers.usda.gov/webdocs/DataFiles/48747/PopulationEstimates.csv?v=3011.3",
    encoding="cp1251",
)

# FILTER OUT STATES AND US POPULATIONS
pop = pop.loc[pop["Rural-urban_Continuum Code_2003"].notnull()]

# CONVIRT FIPS INTS TO 5-DIGIT CODES
pop.loc[:, "FIPS"] = pop.loc[:, "FIPS"].apply(clean_fips)
pop.rename(columns={"FIPS": "fips"}, inplace=True)
pop.loc[:, "fips"] = pop.fips.apply(lambda x: "00000" if x in NYC_FIPS else x)

pop = pop.loc[:, ["POP_ESTIMATE_2018", "fips"]].copy()
pop.loc[:, "POP_ESTIMATE_2018"] = pop["POP_ESTIMATE_2018"].apply(
    lambda x: int(x.replace(",", ""))
)
# MAKE NYC GEOGRAPHY a total of 8million plus
pop = pop.groupby("fips").sum().reset_index()

In [ ]:
# tiger shape file https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/
shapefile = gpd.read_file("shapefile/tl_2019_us_county/tl_2019_us_county.shp")
# convert lat and long to decimal
shapefile.loc[:, "INTPTLAT"] = shapefile.INTPTLAT.apply(lambda x: float(x[1:]))
shapefile.loc[:, "INTPTLON"] = shapefile.INTPTLON.apply(float)

# make fips code 5 digit string
shapefile.loc[:, "fips"] = shapefile.GEOID.apply(
    lambda x: "00000" if x in NYC_FIPS else x
)

In [ ]:
# MAKE NYC into a single geography
shapefile = shapefile.dissolve(by="fips")

In [ ]:
shapefile.reset_index(inplace=True)

In [ ]:
# primitive continental US filter
shapefile = shapefile.loc[
    (shapefile["INTPTLAT"] > 25)
    & (shapefile["INTPTLON"] < 0)
    & (shapefile["INTPTLON"] > -130)
]

In [ ]:
# NY TIMES data updated daily
covid_df = pd.read_csv(
    "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
)
covid_df.loc[:, "fips"] = covid_df["fips"].apply(clean_fips)
covid_df.loc[covid_df["county"] == "New York City", "fips"] = "00000"
covid_df = covid_df.merge(pop, how="left", on="fips")
covid_df.loc[:, "cases_per_100000"] = (
    covid_df.cases / covid_df.POP_ESTIMATE_2018 * 100000
)
covid_df.loc[:, "deaths_per_100000"] = (
    covid_df.deaths / covid_df.POP_ESTIMATE_2018 * 100000
)

In [ ]:
state_shapefile = gpd.read_file("shapefile/tl_2019_us_state/tl_2019_us_state.shp")

state_shapefile.loc[:, "INTPTLAT"] = state_shapefile.INTPTLAT.apply(
    lambda x: float(x[1:])
)
state_shapefile.loc[:, "INTPTLON"] = state_shapefile.INTPTLON.apply(float)


state_shapefile = state_shapefile.loc[
    (state_shapefile["INTPTLAT"] > 25)
    & (state_shapefile["INTPTLON"] < 0)
    & (state_shapefile["INTPTLON"] > -130)
]

In [ ]:
# SHORTEN NAMES FOR TRISTATE MAP

STATES = {"36": "NY", "09": "CT", "34": "NJ"}

shapefile["STATEFP"].apply(lambda x: STATES.get(x, ""))

shapefile.loc[:, "STATE"] = shapefile["STATEFP"].apply(lambda x: STATES.get(x, ""))

In [ ]:
# filter shapefiles for different maps
TRI_STATE = ["09", "34", "36"]
NORTH_EAST = ["25", "00", "33", "34", "36", "42", "50", "09", "23", "44"]

shapefiles = {
    "tristate": [
        shapefile.loc[shapefile.STATEFP.isin(TRI_STATE)],
        state_shapefile.loc[state_shapefile.STATEFP.isin(TRI_STATE)],
    ],
    "us": [shapefile, state_shapefile],
    "northeast": [
        shapefile.loc[shapefile.STATEFP.isin(NORTH_EAST)],
        state_shapefile.loc[state_shapefile.STATEFP.isin(NORTH_EAST)],
    ],
}

In [ ]:
metadata = {
    "cases": {
        "variable": "cases_per_100000",
        "vmin": 0,
        "vmax": 300,
        "title": "Confirmed COVID-19 Cases per 100,000 residents",
        'threshold': 300,
        
    },
    "deaths": {
        "variable": "deaths_per_100000",
        "vmin": 0,
        "vmax": 10,
        "title": "Confirmed COVID-19 Deaths per 100,000 residents",
        "threshold": 10
        
    }
}

In [ ]:
for key, meta in metadata.items():
    variable = meta.get('variable')
    vmin = meta.get('vmin')
    vmax = meta.get('vmax')
    title = meta.get('title')
    threshold = meta.get('threshold')

    for output_path, v in shapefiles.items():
        output_path = f"maps/{key}/{output_path}"
        os.makedirs(output_path, exist_ok=True)

        ct, st = v

        for date in sorted(
            covid_df.loc[covid_df.date >= "2020-03-11", "date"].unique(), reverse=True
        ):

            # this will save the figure as a high-res png in the output path. you can also save as svg if you prefer.
            filepath = os.path.join(output_path, f"{date}.png")
            if not os.path.exists(f"{os.getcwd()}/{filepath}"):
                # merge shapefile with daily covid data
                merged = ct.merge(covid_df.loc[covid_df["date"] == date], how="left", on="fips")
                # fill nans for counties without cases
                merged[variable].fillna(0, inplace=True)
                counties = merged.plot(
                    column=variable,
                    cmap="Reds",
                    figsize=(16, 10),
                    linewidth=0.8,
                    edgecolor="0.8",
                    vmin=vmin,
                    vmax=vmax,
                    legend=True,
                    norm=plt.Normalize(vmin=vmin, vmax=vmax),
                )

                fig = st.geometry.boundary.plot(
                    color=None, edgecolor="k", linewidth=0.5, ax=counties
                )  # Use your second dataframe
                fig.axis("off")

                # add a title
                fig.set_title(
                    title,
                    fontdict={"fontsize": "25", "fontweight": "3"},
                )

                # position the annotation to the bottom left
                date_str = datetime.datetime.strptime(date, "%Y-%m-%d").strftime("%B %d, %Y")
                fig.annotate(
                    date_str,
                    xy=(0.1, 0.225),
                    xycoords="figure fraction",
                    horizontalalignment="left",
                    verticalalignment="top",
                    fontsize=35,
                )
                fig.annotate(
                    "Source: The New York Times - github.com/nytimes/covid-19-data",
                    xy=(0.1, 0.155),
                    xycoords="figure fraction",
                    horizontalalignment="left",
                    verticalalignment="top",
                    fontsize=10,
                )
#             if ("us" not in output_path):
                # counties with more than 100 cases per 100,000
                cases = ""
                for x in (
                    merged.loc[
                        merged[variable] > threshold, ["state", "county", variable]
                    ]
                    .sort_values(by=variable, ascending=False)
                    .values
                ):
                    if key == 'cases':
                        cases += f"{x[0]}-{x[1]}: {int(x[2])}+\n"
                    else:
                        cases += f"{x[0]}-{x[1]}: {round(x[2], 3)}+\n"
                fig.annotate(
                    cases,
                    xy=(0.85, 0.875),
                    xycoords="figure fraction",
                    horizontalalignment="left",
                    verticalalignment="top",
                    fontsize=8,
                )

                chart = fig.get_figure()

                chart.savefig(filepath, dpi=100)
#         plt.close(chart)

In [36]:
covid_df.loc[(covid_df['state'] == 'Georgia') & (covid_df.date == '2020-04-12'), ].sort_values(by='deaths', ascending=False)

,date,county,state,fips,cases,deaths,POP_ESTIMATE_2018,cases_per_100000,deaths_per_100000
51571,2020-04-12,Dougherty,Georgia,13095,1178,72,91243.0,1291.057944,78.910163
51584,2020-04-12,Fulton,Georgia,13121,1495,50,1050114.0,142.365496,4.761388
51557,2020-04-12,Cobb,Georgia,13067,728,35,756865.0,96.186242,4.624339
51590,2020-04-12,Gwinnett,Georgia,13135,701,19,927781.0,75.556624,2.047897
51532,2020-04-12,Bartow,Georgia,13015,213,16,106408.0,200.172919,15.036463
51611,2020-04-12,Lee,Georgia,13177,244,15,29764.0,819.782287,50.396452
51567,2020-04-12,DeKalb,Georgia,13089,893,14,756558.0,118.034572,1.850486
51624,2020-04-12,Mitchell,Georgia,13205,138,14,22192.0,621.845710,63.085797
51555,2020-04-12,Clayton,Georgia,13063,372,11,289615.0,128.446386,3.798146
51553,2020-04-12,Clarke,Georgia,13059,84,11,127330.0,65.970313,8.638970


In [ ]:
!for i in maps/cases/tristate/*.png; do sips -s format jpeg -s formatOptions 70 "${i}" --out "${i%png}jpg"; done

# use jpgs to make gif using imagemagick
!convert -delay 60 -loop 0 maps/cases/tristate/*.jpg maps/cases/tristate/covid.gif

!for i in maps/cases/northeast/*.png; do sips -s format jpeg -s formatOptions 70 "${i}" --out "${i%png}jpg"; done
    

# use jpgs to make gif using imagemagick
!convert -delay 60 -loop 0 maps/cases/northeast/*.jpg maps/cases/northeast/covid.gif

!for i in maps/cases/us/*.png; do sips -s format jpeg -s formatOptions 70 "${i}" --out "${i%png}jpg"; done
    

# use jpgs to make gif using imagemagick
!convert -delay 60 -loop 0 maps/cases/us/*.jpg maps/cases/us/covid.gif

In [ ]:
!for i in maps/deaths/tristate/*.png; do sips -s format jpeg -s formatOptions 70 "${i}" --out "${i%png}jpg"; done

# use jpgs to make gif using imagemagick
!convert -delay 60 -loop 0 maps/deaths/tristate/*.jpg maps/deaths/tristate/covid.gif

!for i in maps/deaths/northeast/*.png; do sips -s format jpeg -s formatOptions 70 "${i}" --out "${i%png}jpg"; done
    

# use jpgs to make gif using imagemagick
!convert -delay 60 -loop 0 maps/deaths/northeast/*.jpg maps/deaths/northeast/covid.gif

!for i in maps/deaths/us/*.png; do sips -s format jpeg -s formatOptions 70 "${i}" --out "${i%png}jpg"; done
    

# use jpgs to make gif using imagemagick
!convert -delay 60 -loop 0 maps/deaths/us/*.jpg maps/deaths/us/covid.gif